In [1]:
import pandas as pd
import re
import sys

In [6]:
#Modulo que  agrega el path de los modulos al sys.path
from pathlib import Path

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()

# Agregar la ruta al sys.path
if str(ruta_modulos) not in sys.path:
    sys.path.append(str(ruta_modulos))          

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()
print(f"Ruta de modulos: {ruta_modulos}")  # Verifica la ruta que se obtiene

Ruta de modulos: C:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\modulos


In [7]:
from utilidades.pandas_funciones import cargar_datos
# Carga de datos
df_proveedor = cargar_datos("prueba.csv")
df_productos = cargar_datos("L_STMaestroPrd.csv")

ImportError: cannot import name 'cargar_datos' from 'utilidades.pandas_funciones' (C:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\modulos\utilidades\pandas_funciones.py)

In [18]:
df_productos['atrib0']

0                  JAYOR
1                    NaN
2                    NaN
3                 ETILEN
4                 ETILEN
              ...       
56769    LKM ONCO/ESPECI
56770    LKM ONCO/ESPECI
56771             PFIZER
56772             PFIZER
56773                ISA
Name: atrib0, Length: 56774, dtype: object

In [19]:
df_proveedor

,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO
0,7791939000089,ASPIRINA 100\r\nEstuche: 10 blisters x 10 comp...,"$4.904,38","$2.305,06",53%,50
1,7791939001116,CARDIO VENT3 30 comp masticables x 100 mg,"$1,31",$393,70%,50
2,7791939000096,CALCIO BASE\r\nEstuche: 3 blisters x 20 comp. ...,"$12.995,10","$6.107,70",53%,90
3,7791939000133,CARBON VEGETAL ACTIVADO\r\nEstuche: 10 blister...,"$14.071,69","$6.613,69",53%,50
4,7791939001383,CLONAZEPAM SL ...,"$ 6.208,60","$ 1.738,41",72%,40
5,7791939001390,CLONAZEPAM ...,"$ 11.824,64","$ 3.310,90",72%,40
6,7791939001413,CLONAZEPAM ...,"$ 11.326,00","$ 3.171,28",72%,40
7,7791939001437,CLONAZEPAM ...,"$ 12.593,93","$ 3.526,30",72%,40
8,7791939000416,IBUPROFENO 400\r\nEstuche: 2 blisters x 10 com...,"$3.100,34","$868,10",72%,50
9,7791939000522,IBUPROFENO 600\r\nEstuche: 2 blisters x 10 com...,"$6.567,98","$1.839,03",72%,40


In [20]:
#1. Solicitar el valor que se desea buscar al usuario 
laboratio_input = input("Ingrese el nombre del laboratorio: ").strip()

In [21]:
#Funcion Busqueda de Laboratorio
def laboratio_input_df(df, columna, valor_buscado):

    """
    Busca coincidencias parciales de un valor en una columna de un DataFrame.

    Parámetros:
    df (DataFrame): El DataFrame donde se buscarán las coincidencias.
    columna (str): El nombre de la columna en la que se realizará la búsqueda.
    valor_buscado (str): El valor que se desea buscar en la columna.

    Retorna:
    DataFrame: Tabla con las coincidencias encontradas.
    """

    #Limpiar el valor buscado
    input_laboratorio= valor_buscado.strip()

    #Buscar coincidencias parciales en la columna especificada.
    tablas_coincidencias = df[df[columna].str.contains(valor_buscado, case=False, na=False)]

    #Verificar Coincidencias 
    if not tablas_coincidencias.empty:
        print("Tabla con valores coincidentes")
        return tablas_coincidencias
    else:
        print("No se encontraron coincidencias")
        return None

In [22]:
#2. Llamar la función con el DataFrame, el nombre de la columna y el valor ingresado
nombre_columna = 'atrib0'
tablas_coincidencias = laboratio_input_df(
    df_productos, 
    nombre_columna, 
    laboratio_input)

Tabla con valores coincidentes


In [23]:
## 3. Si la función encuentra coincidencias, las imprimirá 
tablas_coincidencias

,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,atrib0,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9
416,18530.0,20113,20113,ACICLOVIR MARTIAN 200 mg caps.x 20,NaN,ACICLOVIR MARTIAN 20,PT,7795367001441,NaN,UN,...,LKM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,18531.0,20114,20114,ACICLOVIR MARTIAN 200 mg caps.x 40,NaN,ACICLOVIR MARTIAN 20,PT,NaN,NaN,UN,...,LKM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,18532.0,20117,20117,ACICLOVIR MARTIAN 250 mg iny.liof.a.x 5,NaN,ACICLOVIR MARTIAN 25,PT,7795367001427,NaN,UN,...,LKM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,18533.0,20118,20118,ACICLOVIR MARTIAN 500 mg a.x 1,NaN,ACICLOVIR MARTIAN 50,PT,7795367001434,NaN,UN,...,LKM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,18534.0,20115,20115,ACICLOVIR MARTIAN 800 mg comp.x 20,NaN,ACICLOVIR MARTIAN 80,PT,7795367001458,NaN,UN,...,LKM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56540,118008.0,28483,28483,ZIDOVUDINA LKM 200 mg/20 ml iny.f.a.x 1,NaN,ZIDOVUDINA LKM 200 m,PT,7795367550437.0,NaN,UN,...,LKM ONCO/ESPECI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56569,18751.0,53280,53280,ZITAT 60 mg f.a.x 1+diluy.,NaN,ZITAT 60 mg f.a.x 1+,PT,7795367009805.0,NaN,UN,...,LKM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56597,118009.0,49232,49232,ZOLENICO 4 mg iny.liof.x 1+solv.,NaN,ZOLENICO 4 mg iny.li,PT,7795367002868.0,NaN,UN,...,LKM ONCO/ESPECI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56769,118010.0,49702,49702,ZYVALIX 250 mg comp.x 120,NaN,ZYVALIX 250 mg comp.,PT,7795367001885.0,LKM_B,UN,...,LKM ONCO/ESPECI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
